# Checking to see if we can build classifiers

In [1]:
# import transformers and get the tokenizer and model for bert, and move the whole thing to mps
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()
model.to('mps')



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [3]:
import pandas as pd
df = pd.read_csv('/Users/ananthmuppidi/IIIT_H/semester_6/RSAI/project/textfooler-dataset.csv')  

In [ ]:
# train the bert model on the dataset 

# get the embeddings for the dataset
embeddings = []
for i in range(len(df)):
    text = df.iloc[i, 0]
    input_ids = tokenizer.encode(text, return_tensors='pt')
    with torch.no_grad():
        last_hidden_states = model(input_ids)[0]
    embeddings.append(last_hidden_states[0][0].numpy())

embeddings = np.array(embeddings)
np.save('embeddings.npy', embeddings)

